<a href="https://colab.research.google.com/github/welcomeglory/Python/blob/master/wine_red_white.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install tqdm
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Define the model class
class WineANN(tf.keras.Model):
    def __init__(self):
        super(WineANN, self).__init__()

        self.hid1 = tf.keras.layers.Dense(32,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop1 = tf.keras.layers.Dropout(0.2)

        self.hid2 = tf.keras.layers.Dense(64,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop2 = tf.keras.layers.Dropout(0.2)

        self.hid3 = tf.keras.layers.Dense(32,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop3 = tf.keras.layers.Dropout(0.2)

        self.outPut = tf.keras.layers.Dense(2,
                                            activation=None,
                                            kernel_initializer=self.weight_initialization(),
                                            bias_initializer=self.weight_initialization())

    def weight_initialization(self):
        return tf.keras.initializers.RandomNormal(mean=0., stddev=1.)

    def call(self, x, training=False):  # Forward pass
        y1 = self.hid1(x)
        d1 = self.drop1(y1, training=training)
        y2 = self.hid2(d1)
        d2 = self.drop2(y2, training=training)
        y3 = self.hid3(d2)
        d3 = self.drop3(y3, training=training)
        logits = self.outPut(d3)
        sy = tf.nn.softmax(logits)
        return logits, sy

# Loss function
@tf.function
def cross_entropy_loss(logits, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# Optimizer and backward pass
learningRate = 0.01
optimizer = tf.optimizers.Adagrad(learningRate)

@tf.function
def backward(model, x, y):
    with tf.GradientTape() as grad:
        logits, sy = model(x, training=True)
        loss = cross_entropy_loss(logits, y)
    grads = grad.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Accuracy function
@tf.function
def accuracy(predY, y):
    correct_predictions = tf.equal(tf.argmax(predY, axis=1), tf.argmax(y, axis=1))
    acc = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return acc

# Training function
def train_learning(epochs, frequency, trainDataIter, model, XTest, yTest):
    y_loss = []
    for epoch in tqdm(range(epochs)):
        bx, by = next(trainDataIter)
        loss = backward(model, bx, by)
        y_loss.append(loss)
        if epoch % frequency == 0:
            predY, sy = model(bx, training=False)
            acc = accuracy(predY, by)
            print(f"Epoch: {epoch+1} ===> Loss: {loss:.4f}, Accuracy: {acc:.4f}")
    predY, sy = model(XTest, training=False)
    acc = accuracy(predY, yTest)
    print(f"Test Accuracy: {acc:.4f}")
    return y_loss

2.17.0


In [2]:
red_path = "/content/drive/MyDrive/세종교육/winequality-red.csv"
white_path = "/content/drive/MyDrive/세종교육/winequality-white.csv"
red_df = pd.read_csv(red_path, sep=';')
red_df['label'] = 0
white_df = pd.read_csv(white_path, sep=';')
white_df['label'] = 1
wine_df = pd.concat([red_df, white_df])
wine_df.reset_index(drop=True, inplace=True)
matrix = wine_df.values
X = matrix[:, :-1]
X = (X-X.min(axis=0)) / (X.max(axis=0)-X.min(axis=0))
print(X)
y = matrix[:, -1]


XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.3, random_state=42)
XTrain, XTest = XTrain.astype('float32'), XTest.astype('float32')

# One-hot encode y
yTrain = tf.one_hot(yTrain.flatten(), 2)
yTest = tf.one_hot(yTest.flatten(), 2)

train_data = tf.data.Dataset.from_tensor_slices((XTrain, yTrain))
train_data = train_data.shuffle(buffer_size=XTrain.shape[0]).batch(300).repeat()
trainDataIter = iter(train_data)
# bx, by = next(trainDataIter)
# print(bx)
# print(by)

epochs = 5000
frequency = 150

model = WineANN()
y_loss = train_learning(epochs, frequency, trainDataIter, model, XTest, yTest)

# # Plot loss curve
# plt.plot(y_loss)
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Loss Curve')
# plt.show()

[[0.29752066 0.41333333 0.         ... 0.19101124 0.20289855 0.33333333]
 [0.33057851 0.53333333 0.         ... 0.25842697 0.26086957 0.33333333]
 [0.33057851 0.45333333 0.02409639 ... 0.24157303 0.26086957 0.33333333]
 ...
 [0.2231405  0.10666667 0.11445783 ... 0.13483146 0.20289855 0.5       ]
 [0.14049587 0.14       0.18072289 ... 0.08988764 0.69565217 0.66666667]
 [0.18181818 0.08666667 0.22891566 ... 0.05617978 0.55072464 0.5       ]]


  0%|          | 15/5000 [00:03<14:36,  5.69it/s] 

Epoch: 1 ===> Loss: 128.1373, Accuracy: 0.2433


  3%|▎         | 170/5000 [00:05<00:45, 106.69it/s]

Epoch: 151 ===> Loss: 6.4265, Accuracy: 0.8967


...

Epoch: 4651 ===> Loss: 0.3391, Accuracy: 0.9767


 97%|█████████▋| 4843/5000 [00:33<00:00, 274.81it/s]

Epoch: 4801 ===> Loss: 0.2753, Accuracy: 0.9867


100%|██████████| 5000/5000 [00:33<00:00, 148.38it/s]

Epoch: 4951 ===> Loss: 0.3890, Accuracy: 0.9833
Test Accuracy: 0.9790


In [ ]:
path = '/content/drive/MyDrive/세종교육/student_grade/'
model.save(path + "wineAnn_tf_2_15_version")
model.summary()

In [4]:
# model1 =tf.keras.models.load_model(path + "wineAnn_tf_2_15_version0")
print(XTest[71:72])
predY = model.predict(XTest[71:72])
print(int(tf.argmax(yTest[71:72][0], axis=0)))
print(predY[1].argmax())
predY

[[0.29752067 0.34666666 0.15662651 0.02300614 0.12292359 0.05555556
  0.19585253 0.17447464 0.44186047 0.19101124 0.26086956 0.5       ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
0
0


(array([[17.843185, 14.863704]], dtype=float32),
 array([[0.9516385 , 0.04836148]], dtype=float32))